In [ ]:
nyc_ts_load

In [ ]:
# Load Remaining Components

# T = pg.T
# F = pg.F

_db = "system"
T.eng                                 =   T.create_engine(r'postgresql://%s:%s@%s:%s/%s'
                                                          %(T.DB_USER,T.DB_PW,T.DB_HOST,T.DB_PORT,_db),
                                                          encoding='utf-8',
                                                          echo=False)

%pdb off

%sql postgresql://$T.DB_USER:$T.DB_PW@$T.DB_HOST:$T.DB_PORT/$_db

In [ ]:
# LOAD BASIC PGSQL FUNCTIONS

get metadata
try to get text content
if no content, then 
echo "GET /send?uuid=%s&filename=%s&filepath=%s" | socat -v - unix:/var/sockets/fsp_send.sock 

echo "GET /send?uuid=06893ea&filename=06893ea&filepath=/home/ub2/ARCHIVE/gmail_attachments/06893ea" | socat -v - unix:/var/sockets/fsp_send.sock

### GENERAL FLOW

1. email added to local server
2. info for each attachment added to `file_idx` table
3. each new file triggers content and metadata extraction
    - pdf files needing OCR are added to file_server queue

#### PDF OCR Flow
1. pgsql uses linux at/batch and queues script to run on PDF
2. script communicates pdf via tcp/sock/websocket to designated file processing server (FPS) and end upon receipt confirmation
    -unix:/var/sockets/fsp_send.sock
3. on FPS, pdf dropped into folder watched by pypdfocr, which creates ocr version and queues script to update pgsql
4. the FPS-queued scripts communicate PDF content back to pgsql via nginx websocket

In [ ]:
%%sql postgres@system
do language plpythonu 
    $BODY$

    from subprocess                         import Popen            as sub_popen
    from subprocess                         import PIPE             as sub_PIPE


    qry = """
        SELECT row_to_json(t) res
        FROM (select * 
            from (
                SELECT * FROM gmail
                WHERE uid = 154934
            ) f
        ) t
          """
    with open('/tmp/tmp','w') as f: f.write(plpy.execute(qry)[0]["res"])
    
    cmd = [ 'echo "GET /json?gmail=%s&uid=154934"' % plpy.execute(qry)[0]["res"],
            ' | socat - tcp:0.0.0.0:12501,reuseaddr,nonblock' ]
    #cmd = [ 'echo "GET /json?gmail=%s"' % plpy.execute(qry)[0]["res"],
    #        ' > /tmp/tmp' ]
    (_out,_err) = sub_popen(''.join(cmd),stdout=sub_PIPE,shell=True).communicate()
    assert _err is None



    $BODY$;

In [ ]:
fs_tbl_and_func="""
    DROP TABLE IF EXISTS file_idx;

    CREATE TABLE file_idx (
        src_db                text,
        src_uid                integer,
        _key                text,
        _filetype            text,
        _info                jsonb,
        _run_ocr                boolean default false,
        _metadata                jsonb,
        _content  text
    );

    create EXTENSION plsh with version '3';

    """
pdf_metadata_func="""
    drop function if exists zf_pdf_metadata( text ) cascade;
    create function zf_pdf_metadata( text )
    returns text as E'
    #!/bin/bash
    pdfinfo -meta $1 | jq -sRcMj \\'[ splits("\n")? | split(":") as $i | 
    { ($i[0]?) : ( $i[1] | sub("( )+"; ""; "sl") ) } ]\\'
    ' LANGUAGE plsh;

    """
pdftotext_func_sh="""
    drop function if exists zf_pdftotext( text ) cascade;
    create function zf_pdftotext( text )
    returns text as E'
    #!/bin/bash
    pdftotext -q -nopgbrk -raw $1 -
    ' LANGUAGE plsh;
    """
pdftotext_func_py="""
    drop function if exists zf_pdftotext( text ) cascade;
    create function zf_pdftotext( text )
    returns text as E'
    #!/bin/bash
    pdftotext -q -nopgbrk -raw $1 -
    ' LANGUAGE plsh;
    """

In [ ]:
%sql postgres@system delete from file_idx where true

In [ ]:
%%sql postgres@system
-- UPDATE/SELECT RECENT UPDATES WITH ATTACHMENTS
    with upd as (
        select 
            uid,
            _attachments a,
            json_array_length(_attachments::json) len
        from (
            select 
                orig_msg->'attachments' _attachments,
                uid
            from gmail
            where json_array_length((orig_msg->'attachments')::json) > 0
            AND (orig_msg->'attachments'->0->(select * from jsonb_object_keys(orig_msg->'attachments'->0) limit 1)
                    ->'content_type')::text ilike '%pdf%'
            AND to_timestamp(((orig_msg->'sent_at')::text)::integer) > date '2015-01-01'
            order by to_timestamp(((orig_msg->'sent_at')::text)::integer) desc
            limit 10

        ) f1
    )
    --select * from upd
    update gmail g set last_updated = now()
    from upd u
    where u.uid = g.uid
    

In [ ]:
%%sql postgres@system
-- SELECT ATTACHMENT INFO WITH GMAIL UID
    select
        'gmail' src_db,
        uid src_uid,
        _key,
        regexp_replace(((_json->_key)::jsonb->'name')::text,E'"(.*)\.([^\.]+)"',E'\2','g') _filetype,
        (_json->_key)::jsonb _info,
        false _run_ocr
    from  (
        select 
            json_object_keys(json_array_elements(_att::json)::json) _key,
            json_array_elements(_att::json)::json _json,
            uid
        from

            (
            select 
                _attachments _att,uid
            from (
                select 
                    orig_msg->'attachments' _attachments,
                    uid
                from gmail
                where 
                json_array_length((orig_msg->'attachments')::json) > 0
                and uid = 77753
            ) f1
        ) f2
    ) f3

## Triggers

In [ ]:
%%sql postgres@system

-- BEFORE INSERT/UPDATE gmail/file_idx, UPDATE elasticsearch
            
DROP FUNCTION if exists fs_update_es() cascade;
DROP TRIGGER if exists fs_update_es_trigger ON gmail;
DROP TRIGGER if exists fs_update_es_trigger ON file_idx;
--CREATE OR REPLACE FUNCTION fs_update_es()
    --    RETURNS TRIGGER as $funct$
    --
    --        _t=trigger
    --        package.path=package.path..';/home/ub2/BD_Scripts/Lua/pgsql/?.lua;;'
    --        local f = require"pg_file_server"
    --        f.update_es_trigger(_t)
    --
    --    $funct$ LANGUAGE plluau;

CREATE OR REPLACE FUNCTION fs_update_es()
    RETURNS TRIGGER as E'#!/bin/bash

        /home/ub2/SERVER2/file_server/ws_elasticsearch.bash "$@"
    
    ' LANGUAGE plsh;

CREATE TRIGGER fs_update_es_trigger
    BEFORE INSERT or UPDATE ON gmail
    FOR EACH ROW
    EXECUTE PROCEDURE fs_update_es();
            
CREATE TRIGGER fs_update_es_trigger
    BEFORE INSERT or UPDATE ON file_idx
    FOR EACH ROW
    EXECUTE PROCEDURE fs_update_es();

In [ ]:
%%sql postgres@system

-- AFTER gmail INSERT/UPDATE, UPDATE file_idx

DROP FUNCTION if exists fs_update_file_idx_on_gmail_update() cascade;
CREATE OR REPLACE FUNCTION fs_update_file_idx_on_gmail_update()
    RETURNS TRIGGER AS $funct$

    T = {'gmail_uid' : TD["new"]["uid"]}

    enumerate_attachments = """
            with upd as (
                select
                    'gmail' src_db,
                    uid src_uid,
                    _key,
                    regexp_replace(((_json->_key)::jsonb->'name')::text,E'"(.*)\\\\.([^\\.]+)"',E'\\\\2','g') _filetype,
                    (_json->_key)::jsonb _info
                from  (
                    select 
                        json_object_keys(json_array_elements(_att::json)::json) _key,
                        json_array_elements(_att::json)::json _json,
                        uid
                    from

                        (
                        select 
                            _attachments _att,uid
                        from (
                            select 
                                orig_msg->'attachments' _attachments,
                                uid
                            from gmail
                            where 
                            json_array_length((orig_msg->'attachments')::json) > 0
                            and uid = %(gmail_uid)s
                        ) f1
                    ) f2
                ) f3
            )
            insert into file_idx (
                src_db,
                src_uid,
                _key,
                _filetype,
                _info
                )
            select 
                u.src_db,
                u.src_uid,
                u._key,
                lower(u._filetype),
                u._info
            from 
                upd u,
                (select array_agg(_key) all_keys from file_idx) s1
            where all_keys is null
            OR not u._key = any(all_keys)

          """ % T
    
    plpy.execute(enumerate_attachments)
    
    $funct$ 
    language "plpythonu";

    
DROP TRIGGER if exists fs_update_file_idx_on_gmail_update_trigger ON gmail;
    
CREATE TRIGGER fs_update_file_idx_on_gmail_update_trigger
    AFTER UPDATE OR INSERT ON gmail
    FOR EACH ROW
    EXECUTE PROCEDURE fs_update_file_idx_on_gmail_update();

In [ ]:
%%sql postgres@system

-- AFTER file_idx INSERT/UPDATE, IDENTIFY/INITIATE FILE PROCESSING

DROP FUNCTION if exists fs_process_files_in_file_idx() cascade;
DROP TRIGGER if exists fs_process_files_in_file_idx_trigger ON file_idx;
CREATE OR REPLACE FUNCTION fs_process_files_in_file_idx()
    RETURNS TRIGGER AS $funct$

    from subprocess                         import Popen            as sub_popen
    from subprocess                         import PIPE             as sub_PIPE
    from traceback                      import format_exc       as tb_format_exc
    from sys                            import exc_info         as sys_exc_info

    def txt_log(msg):
        with open('/tmp/tmp','a+') as f:
            f.write(msg+'\n')

    def upd_pdf(f_path):
        cmd = ';'.join([  'pdfinfo -meta %s | jq -sRcMj \'[ splits("\\n")? | split(":") as $i | \
                                { ($i[0]?) : ( $i[1] | sub("( )+"; ""; "sl") ) } ]\' \
                                | jq -cMj \'add\'' % f_path,
                           'echo "<BREAK>"',
                           'pdftotext -q -nopgbrk -layout %s -' % f_path,
                           'echo "<BREAK>"'
                       ])
        (_out,_err)                 =   sub_popen(cmd,stdout=sub_PIPE,shell=True).communicate()
        assert _err is None
        res = _out.split('<BREAK>')
        TD["new"]["_metadata"] = "'" + str(res[0].strip('\n[]')) + "'"
        _content = res[1].strip('\n')
        if _content:
            TD["new"]["_content"] = "'" + str(_content) + "'"
        else:
            TD["new"]["_run_ocr"] = True
        return TD["new"]

    def upd_from_ocr_pdf(f_path):
        cmd = 'pdftotext -q -nopgbrk -raw %s -' % ('%s_ocr.pdf' % f_path)
        (_out,_err)                 =   sub_popen(cmd,stdout=sub_PIPE,shell=True).communicate()
        assert _err is None
        if _out:
            TD["new"]["_content"] = _out
        return TD["new"]

    def upd_db(TD,orig):
        T = TD["new"]
        _repl = ' '.join(['%s=%s,' % (it,'%('+it+')s') for it in T.keys() if T[it]!=orig[it]])
        qry = "update file_idx set "+_repl[:-1]+' where uid='+str(T['uid'])
        # plpy.log(qry % T)
        plpy.execute(qry % T)
    
    try:
        orig = dict(zip(TD["new"].keys(),TD["new"].values()))
        base_dir = '/home/ub2/ARCHIVE/gmail_attachments/'
        f_key = TD["new"]["_key"]
        f_name = f_key
        f_path = base_dir + f_name
        f_type = TD["new"]["_filetype"]
        f_meta = TD["new"]["_metadata"]
        f_ocr = TD["new"]["_run_ocr"]

        # FIRST PASS -- ATTEMPT TO EXTRACT
        if f_type=='pdf' and not f_meta:
            # plpy.log("FIRST_PASS")
            TD["new"] = upd_pdf(f_path)
            if TD["new"]["_run_ocr"]==True:

                cmd = [ 'echo "GET /send?uuid=%s' % f_key,
                        '&filename=%s' % f_name,
                        '&filepath=%s' % f_path,
                        '" | socat - tcp:0.0.0.0:12501,reuseaddr,nonblock,shut-down'
                        ]
                (_out,_err) = sub_popen(''.join(cmd),stdout=sub_PIPE,shell=True).communicate()
                assert _err is None


        # SECOND PASS -- EXTRACT FROM OCR PDF
        elif f_meta and not f_ocr:
            # plpy.log("SECOND_PASS")
            TD["new"] = upd_from_ocr_pdf(f_path)

        if TD["new"]!=orig:
            upd_db(TD,orig)

    except plpy.SPIError:
        plpy.log('fps_gmail_update FAILED')
        plpy.log(tb_format_exc())
        plpy.log(sys_exc_info()[0])
        return


    $funct$ 
    language "plpythonu";

CREATE TRIGGER fs_process_files_in_file_idx_trigger
    AFTER UPDATE OR INSERT ON file_idx
    FOR EACH ROW
    EXECUTE PROCEDURE fs_process_files_in_file_idx();

## TESTS

In [ ]:
%%sql postgres@system
-- TEST fs_process_files_in_file_idx_trigger
    with _sel as (
        select * from file_idx
        where _content is null
        order by uid asc
        limit 1
    )
    update file_idx f set last_updated=now()
    from _sel s
    where s.uid = f.uid;

In [ ]:
%%sql postgres@system
-- TEST fs_process_files_in_file_idx_trigger
    update gmail set last_updated=now()
    where uid = 107966

In [ ]:
%%sql postgres@system
-- TEST BATCH fs_process_files_in_file_idx_trigger
    with upd as (
        select src_uid from all_attachments aa
        order by aa.src_uid asc
        offset 10
        limit 1000
    )
    update gmail g set last_updated=now()
    from upd u
    where g.uid = u.src_uid
    

In [ ]:
%%sql postgres@system 
-- MOCK FILE_IDX INSERT
    delete from file_idx where src_uid=128859;

    insert into file_idx (
        src_db,
        src_uid,
        _key,
        _filetype,
        _info,
        _run_ocr
    )
    values (
        'gmail',
        128859,
        '06893ea',
        'pdf',
        '{"name": "LAP.pdf", "content_type": "application/pdf", "size_in_kb": 22254}'::jsonb,
        false
    );

In [ ]:
%%sql postgres@system 
-- SHOW FILE_IDX
    select * from file_idx

In [ ]:
%%sql postgres@system 
-- DELETE FROM FILE_IDX
    DELETE from file_idx WHERE 
    true
    --src_uid=77753

### Create & Test gmail/file_idx INSERT TRIGGERS

In [ ]:
%%sql postgres@system 
-- MOCK GMAIL INSERT
    delete from gmail where msg_id='test';
    insert into gmail (
        g_msg_id,
        msg_id
    )
    values (
        10,
        'test'
    );
-- MOCK FILE_IDX INSERT
    delete from file_idx where _key='text key';
    insert into file_idx (
        _key,
        _filetype
    )
    values (
        'text key',
        'test filetype'
    );